# Transfer learning

Vamos usar uma CNN treinada em um dataset para resolver uma outra tarefa que possivelmente não tem uma ligação direta com a tarefa original.


## Classificação morfológica de galaxias

Outro problema de classificação de imagens é aquele chamado de **classificação morfológica de galaxias**: precisamos distinguir os tipos de galaxia de a cordo com sua aparência visual.
Pegamos alguns exemplos de imagens em [EFIGI reference dataset](https://www.aanda.org/articles/aa/pdf/forth/aa16423-10.pdf). Há cinco tipos de galaxia:  **elliptical**, **irregular** , **spiral**, **dwarf** e **lenticular**.

<table>
<tr>
<td>
<img align="middle"   width='200' heith='100' src='images/elliptical.png'>
<img align="middle"   width='200' heith='100' src='images/irregular.png'>
<img align="middle"   width='200' heith='100' src='images/spiral.png'>
<img align="middle"   width='200' heith='100' src='images/dwarf.png'>
<img align="middle"   width='200' heith='100' src='images/lenticular.png'>

</td>
</tr>
</table>

In [ ]:
# notebook feito para a versão 0.4.0 do Pytorch 
import torch
import torch.nn as nn
import subprocess
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from plots import plot9images, plot_confusion_matrix, plot_histogram_from_labels
from util import randomize_in_place

Baixando os dados

In [ ]:
# Essa célula pode demorar de acordo com sua conexão de internet.
# Olhe o terminal para mais informações sobre o download
if not os.path.exists("efigi_data"):
    pro = subprocess.Popen(["bash", "download_efigi.sh"])
    pro.wait()

Vamos criar funções para aumentar os dados e normalizá-los.

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


Vamos usar a classe [`ImageFolder`](https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder) do PyTorch para transformar as imagens em tensores e aplicar todas as manipulações.

In [ ]:
data_dir = 'efigi_data'
train_dir = os.path.join(data_dir, "train")
train_data = datasets.ImageFolder(train_dir,transform=data_transforms["train"])
print(train_data)

In [ ]:
test_dir = os.path.join(data_dir, "test")
test_data = datasets.ImageFolder(test_dir,transform=data_transforms["test"])
print(test_data)

In [ ]:
valid_dir = os.path.join(data_dir, "valid")
valid_data = datasets.ImageFolder(valid_dir,transform=data_transforms["val"])
print(valid_data)

Note que nesse caso, temos **poucos** dados de treinamento, teste e validação.

In [ ]:
train_loader = DataLoader(train_data, batch_size=9, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_data, batch_size=9, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=45, num_workers=4)

Vamos definir algumas variáveis globais úteis

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataloaders = {"train": train_loader, "val": valid_loader}
dataset_sizes = {"train": len(train_data), "val": len(valid_data)}

class_names = train_data.classes
print(class_names)
int2label = {k:v for k,v in enumerate(class_names)}

Vamos observar algumas imagens do dataset

In [ ]:
def transform_image(inp):
    out = torchvision.utils.make_grid(inp)
    inp = out.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    return inp


inputs, classes = next(iter(train_loader))

img9 = inputs[0:9]
img9 = np.array([transform_image(i) for i in img9])
labels9 = classes[0:9].numpy()
labels9 = [int2label[i] for i in labels9]
img9 = img9.reshape((9, 224, 224, 3))
img9 = img9[...,::-1]
plot9images(img9, labels9, (224, 224, 3))

Para mudar o lerning rate ao longo do treinamento vamos criar uma outra versão do loop de treinamento.

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0


            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)


                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)


                    if phase == 'train':
                        loss.backward()
                        optimizer.step()


                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

Vamos baixar uma CNN já treinada chamada **resnet18**.

In [ ]:
model_ft = models.resnet18(pretrained=True)

Vamos congelar todos os pesos dessa rede.

In [ ]:
for param in model_ft.parameters():
    param.requires_grad = False

Mudamos a última camada dessa rede para se adptar a nossa tarefa de classificação

In [ ]:
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 5)
model_ft = model_ft.to(device)
print(model_ft.fc)

Para realizar o treinamento vamos definir um função de custo, o otimizafor e como vamos fazer o learning rate decair ao longo do treinamento (aqui estamos usando a classe `lr_scheduler.StepLR`)

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.fc.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

Podemos olhar agora o quão bom está o nosso modelo.

In [ ]:
img, labels = next(iter(test_loader))

pred = model_ft(img)
softmax = nn.Softmax(dim=1)
out = softmax(pred)
_, predictions = torch.max(out, 1) 
predictions = predictions.numpy()

plot_confusion_matrix(truth=labels.numpy(),
                      predictions=predictions,
                      save=False,
                      path="transfer_learning_confusion_matrix.png",
                      classes=class_names)

pred9 = predictions[0:9]
pred9 = [int2label[i] for i in pred9] 
img9 = img[0:9]
img9 = np.array([transform_image(i) for i in img9])
labels9 = labels[0:9].numpy()
labels9 = [int2label[i] for i in labels9]
img9 = img9.reshape((9, 224, 224, 3))
img9 = img9[...,::-1]
plot9images(img9, labels9, (224, 224, 3), pred9)

Nosso resultado não está ótimo. Mas note que estamos treinando um modelo com poucos dados. Use outras redes já treinadas para obter uma acurácia maior.